In [4]:
import cx_Oracle
import numpy as np
import pandas as pd
from faker import Faker
from datetime import datetime, timedelta
from random import randrange
from datetime import timedelta
import random
import time

In [5]:
connection = cx_Oracle.connect(user='scott', password='tiger', dsn='192.168.119.119:1521/dink11.dbsvr')
cursor = connection.cursor()
# cursor.close()

## 지역 인코딩

In [ ]:
# load region
cursor = connection.cursor()
sql="select * from region"
cursor.execute(sql)

rows = cursor.fetchall()
columns = ['regionCode','regionName']
region = pd.DataFrame(rows, columns=columns)

regionCode=region.regionCode.tolist()
seoultToRegionTime=[70,70,130,130,180,180,170,300,290,240,200,250,230,230,300]
seoultToRegionDistance=[40,40,100,131,163,132,152,406,376,300,225,365,327,228,500]

sql = "UPDATE region SET seoultToRegionTime = :1, seoultToRegionDistance = :2 WHERE regioncode = :3"
for i in range(len(regionCode)):
    cursor.execute(sql, (seoultToRegionTime[i],seoultToRegionDistance[i],regionCode[i]))
    
connection.commit()
cursor.close()

## 업종 인코딩

### FastText Pre-trained 한국어 모델 사용
1. FastText 소개
FastText는 Facebook에서 만든 word representation 과 sentence classification의 효율적인 학습을 위한 라이브러리
2. Fast Text 설치 및 pre-trained 모델 다운받기
3. Pre-trained 한국어 모델 적용

In [ ]:
from gensim import models

ko_model = models.fasttext.load_facebook_model("C:/Users/pooh5/Downloads/cc.ko.300-001.bin")
for w, sim in ko_model.wv.similar_by_word('파이썬', 10):
    print(f'{w}: {sim}')
    
ko_model.wv.vectors.shape

# 한글 폰트 사용을 위해서 세팅
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)


In [ ]:
# load category
cursor = connection.cursor()
sql="select * from category"
cursor.execute(sql)

rows = cursor.fetchall()
cursor.close()
columns = ['categoryCode','categoryBig','categoryMiddle','categorySmall']
category = pd.DataFrame(rows, columns=columns)

# 단어들 간에 유사도 확인
# 유흥업소 기준 encoding
words = category['categorySmall'].tolist()[:]
word_embedding = []
for w in words:
    # print('유흥업소',str(w),': ',ko_model.wv.similarity('유흥업소', w))
    word_embedding.append(ko_model.wv.similarity('유흥업소', w))
category['barToEmbedding'] = word_embedding
category.loc[category['categorySmall'] == '기타유흥업소', 'barToEmbedding'] = 1

# 단어들 간에 유사도 확인
# 귀금속 기준 encoding
words = category['categorySmall'].tolist()[:]
word_embedding = []
for w in words:
    # print('귀금속',str(w),': ',ko_model.wv.similarity('귀금속', w))
    word_embedding.append(ko_model.wv.similarity('귀금속', w))
category['diamondToEmbedding'] = word_embedding

categorySmall=category.categorySmall.tolist()
barToEmbedding=category.barToEmbedding.tolist()
diamondToEmbedding=category.diamondToEmbedding.tolist()

cursor = connection.cursor()
sql = "UPDATE category SET barToEmbedding = :1, diamondToEmbedding = :2 WHERE categorySmall = :3"
for i in range(len(categorySmall)):
    cursor.execute(sql, (barToEmbedding[i],diamondToEmbedding[i],categorySmall[i]))
    
connection.commit()
cursor.close()
